In [1]:
# import bus_processing
import bus_prediction
import os
import pandas as pd
from constants import DATA_DIR

In [2]:
def load_bus_data():
    path = os.path.join(DATA_DIR, "dataset.csv")
    df = pd.read_csv(path)
    return df

In [3]:
bus_data = load_bus_data()

In [4]:
bus_data["time_bucket"] = pd.to_datetime(bus_data["time_bucket"], utc=True)

In [5]:
X_train, y_train, X_test, y_test = bus_prediction.prepare_bus_data(bus_data)

In [6]:
lstm_model = bus_prediction.LSTMModule(input_size=X_train.shape[1], hidden_size=64, num_layers=5, dropout=0.1)
lstm_predictor = bus_prediction.LSTMPredictor(lstm_model, window_size=10)

In [7]:
lstm_predictor.train(X_train.values, y_train.values, epochs=20)

/Users/legendword/Documents/zwh/Coding/CPSC440/cpsc440-transit-project/src/bus_prediction.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  self.X = torch.tensor(X_lagged, dtype=torch.float32)


Epoch 1/20, Train Loss: 1.5495, Val Loss: 0.9400
Epoch 2/20, Train Loss: 1.4579, Val Loss: 0.8963
Epoch 3/20, Train Loss: 1.4322, Val Loss: 0.8315
Epoch 4/20, Train Loss: 1.2970, Val Loss: 0.7655
Epoch 5/20, Train Loss: 1.2481, Val Loss: 0.8710
Epoch 6/20, Train Loss: 1.2080, Val Loss: 0.8774
Epoch 7/20, Train Loss: 1.1769, Val Loss: 0.8047
Epoch 8/20, Train Loss: 1.0899, Val Loss: 0.6750
Epoch 9/20, Train Loss: 0.9542, Val Loss: 0.5174
Epoch 10/20, Train Loss: 0.8783, Val Loss: 0.4519
Epoch 11/20, Train Loss: 0.7632, Val Loss: 0.5424
Epoch 12/20, Train Loss: 0.6833, Val Loss: 0.5073
Epoch 13/20, Train Loss: 0.7020, Val Loss: 1.0504
Epoch 14/20, Train Loss: 0.6551, Val Loss: 0.3633
Epoch 15/20, Train Loss: 0.6308, Val Loss: 0.4954
Epoch 16/20, Train Loss: 0.6071, Val Loss: 0.5939
Epoch 17/20, Train Loss: 0.6211, Val Loss: 0.4169
Epoch 18/20, Train Loss: 0.5647, Val Loss: 0.5671
Epoch 19/20, Train Loss: 0.5357, Val Loss: 0.4557
Epoch 20/20, Train Loss: 0.5588, Val Loss: 0.7497


In [ ]:
lstm_predictor.train(bus_train) # old output

/Users/legendword/Documents/zwh/Coding/CPSC440/cpsc440-transit-project/src/bus_prediction.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["avg_delay"] = X["delay_total"] / X["count"]
/Users/legendword/Documents/zwh/Coding/CPSC440/cpsc440-transit-project/src/bus_prediction.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(


Epoch 1/10, Train Loss: 0.7393, Val Loss: 0.2799
Epoch 2/10, Train Loss: 0.4401, Val Loss: 0.2299
Epoch 3/10, Train Loss: 0.4179, Val Loss: 0.2225
Epoch 4/10, Train Loss: 0.4008, Val Loss: 0.2206
Epoch 5/10, Train Loss: 0.3946, Val Loss: 0.2244
Epoch 6/10, Train Loss: 0.3886, Val Loss: 0.2188
Epoch 7/10, Train Loss: 0.3765, Val Loss: 0.2190
Epoch 8/10, Train Loss: 0.3723, Val Loss: 0.2181
Epoch 9/10, Train Loss: 0.3626, Val Loss: 0.2222
Epoch 10/10, Train Loss: 0.3606, Val Loss: 0.2240
